In [12]:
#подключение библиотек
import numpy as np
import time
from datetime import datetime
import plotly.express as px
from typing import Any, Callable, List, Dict
from multiprocessing import Pool, TimeoutError
import ipyparallel as ipp

In [13]:
# 100 массивов с 10 элементами от 1 до 10
arr10 = [np.random.randint(low=1, high=10, size=10) for i in range(100)]
arr10s = [ np.sort(arr10[i], kind="mergesort") for i in range(100)] #отсортированне массивы для проверки

# 100 массивов с 25 элементами от 1 до 25
arr25 = [np.random.randint(low=1, high=25, size=25) for i in range(100)]
arr25s = [ np.sort(arr25[i], kind="mergesort") for i in range(100)]#отсортированне массивы для проверки

# 100 массивов с 500 элементами от 1 до 500
arr5h = [np.random.randint(low=1, high=500, size=500) for i in range(100)]
arr5hs = [ np.sort(arr5h[i], kind="mergesort") for i in range(100)]#отсортированне массивы для проверки

# 100 массивов с 1000 элементами от 1 до 1000
arr1th = [np.random.randint(low=1, high=1000, size=1000) for i in range(100)]
arr1ths = [ np.sort(arr1th[i], kind="mergesort") for i in range(100)]#отсортированне массивы для проверки

# 100 массивов с 5000 элементами от 1 до 5000
arr5th = [np.random.randint(low=1, high=5000, size=5000) for i in range(100)]
arr5ths = [ np.sort(arr5th[i], kind="mergesort") for i in range(100)]#отсортированне массивы для проверки


In [14]:
swap_count = 0

def swap(arr: np.ndarray, a: int, b: int):
    swap_count_change()
    arr[a], arr[b] = arr[b], arr[a]

def swap_count_change():
    global swap_count
    swap_count += 1


In [15]:
#сортировка простыми вставками
def insertion_sort(arr: np.ndarray):
    n = arr.shape[0]
    for i in range(1, n):
        key = arr[i]
        j = i - 1 
        while (j >= 0 and arr[j] > key):
            arr[j + 1] = arr[j]
            swap_count_change()
            j = j - 1
        arr[j + 1] = key
        swap_count_change()

In [16]:
#сортировка простым обменом
def exchange_sort(arr: np.ndarray):
    n = arr.shape[0]
    for i in range(0, n-1):
        for j in range(i+1, n):
            if (arr[i] > arr[j]):
                swap(arr, j, i)

In [17]:
#сортировка простым выбором 
def selection_sort(arr: np.ndarray):
    n = arr.shape[0]
    for i in range(n-1):
        _min = i
        for j in range(i+1, n):
            if (arr[j] < arr[_min]):
                _min = j
        if (_min != i):
            swap(arr, i, _min)

In [18]:
#сортировка пирамидальная
def heapify(arr: np.ndarray, n: int, i: int):
    large = i
    l = 2 * i + 1
    r = l + 1
    if (l < n and arr[i] < arr[l]):
        large = l
    if (r < n and arr[large] < arr[r]):
        large = r
    if large != i:
        swap(arr, i, large)
        heapify(arr, n, large)
    
def heap_sort(arr: np.ndarray):
    n = arr.shape[0]
    
    #построение дерева
    for i in range(n//2, -1, -1):
        heapify(arr, n, i)

    # Один за другим извлекаем элементы
    for i in range(n-1, 0, -1):
        swap(arr, i, 0)
        heapify(arr, i, 0)

In [19]:
#быстрая сортировка
def partition(arr: np.ndarray, start: int, end: int) -> int:
    pivot = arr[(start+end) // 2]
    i=start
    j=end

    while i <=j:
        while arr[i] < pivot:
            i+=1
        while arr[j] > pivot:
            j-=1
        if i <= j:
            swap(arr, i, j)
            i+=1
            j-=1
    return i

def quick_sort_f(arr: np.ndarray, start: int, end: int):
    if start < end:
        temp = partition(arr, start, end)
        quick_sort_f(arr, start, temp - 1)
        quick_sort_f(arr, temp, end)

def quick_sort(arr: np.ndarray):
    quick_sort_f(arr, 0, arr.shape[0] - 1)

In [20]:
#проверка времени выполнения блока данных
def check_function_time(arr: np.ndarray, func: Callable) -> float:
    star_time = time.time()
    func(arr)
    end_time = time.time()
    return (end_time - star_time)

In [18]:
#запуск процессов
def process_start(algorithm: Callable, arr: List[np.ndarray]) -> Dict[str, List[float]]:
    time_arr = []
    with Pool(processes=10) as pool:
        res =  pool.apply_async(check_function_time, [[algorithm, arr[i]]  for i in range(10)])
        async_worker = res.get(timeout=30)
        print('1 процесс, worker(20) => ', async_worker)
#         time_arr = pool.map(check_function_time, [(arr[i], algorithm) for i in range(10)])
#         print(type(time_arr))
    return ({str(len(arr[0])): time_arr})

In [19]:
time_arr = {}
# sort_algorithm = [insertion_sort, exchange_sort, selection_sort, heap_sort, quick_sort]
sort_algorithm = [quick_sort]
data = [arr10, arr25, arr5h, arr1th, arr5th]
for algorithm in sort_algorithm:
    algorithm_name = algorithm.__name__
    time_arr[algorithm_name] = []
    for arr in data:
        time_arr[algorithm_name].append(process_start(algorithm, arr))
    break
print(time_arr)
#         time_arr[algorithm_name].append(process_start(algorithm, arr.copy()))

TimeoutError: 

In [ ]:
def f(x):
    return (x*x)
with Pool(processes=10) as pool:
    print(pool.map(f, [i for i in range(10)]))

In [11]:
def mul(a, b):
    time.sleep(0.5 * random.random())
    return a * b

def plus(a, b):
    time.sleep(0.5 * random.random())
    return a + b
    

In [12]:
 TASKS = [(mul, (i, 7)) for i in range(10)] + \
                [(plus, (i, 8)) for i in range(10)]

In [8]:
d = [t for t in TASKS]


In [14]:
TASKS

[(<function __main__.mul(a, b)>, (0, 7)),
 (<function __main__.mul(a, b)>, (1, 7)),
 (<function __main__.mul(a, b)>, (2, 7)),
 (<function __main__.mul(a, b)>, (3, 7)),
 (<function __main__.mul(a, b)>, (4, 7)),
 (<function __main__.mul(a, b)>, (5, 7)),
 (<function __main__.mul(a, b)>, (6, 7)),
 (<function __main__.mul(a, b)>, (7, 7)),
 (<function __main__.mul(a, b)>, (8, 7)),
 (<function __main__.mul(a, b)>, (9, 7)),
 (<function __main__.plus(a, b)>, (0, 8)),
 (<function __main__.plus(a, b)>, (1, 8)),
 (<function __main__.plus(a, b)>, (2, 8)),
 (<function __main__.plus(a, b)>, (3, 8)),
 (<function __main__.plus(a, b)>, (4, 8)),
 (<function __main__.plus(a, b)>, (5, 8)),
 (<function __main__.plus(a, b)>, (6, 8)),
 (<function __main__.plus(a, b)>, (7, 8)),
 (<function __main__.plus(a, b)>, (8, 8)),
 (<function __main__.plus(a, b)>, (9, 8))]